<a href="https://colab.research.google.com/github/SantiagoPendon/SLICER_RAG/blob/main/SLICER_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu --no-cache-dir
!pip install sentence-transformers --no-cache-dir
!pip install transformers==4.52.4
!pip install protobuf==3.20.3

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            

In [ ]:
# Load the files from Local storage:
from google.colab import files
files.upload()

{}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
import torch
# 1. Modelo base
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16, device_map="auto")

# 2. Configuración LoRA
lora_config = LoraConfig(
    r=16,         # rango bajo para LoRA
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # ejemplo de módulos a adaptar
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

KeyboardInterrupt: 

In [ ]:
model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


In [ ]:
from pathlib import Path

file_paths = list(Path("/content").rglob("*.py")) + \
             list(Path("/content").rglob("*.cxx"))

files_content = {}
for path in file_paths:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        files_content[path.name] = f.read()




In [ ]:
def split_into_chunks(text, chunk_size=50, overlap=10):
    """
    text: string del fichero
    chunk_size: número de líneas por chunk
    overlap: número de líneas que se repiten entre chunks
    """
    lines = text.splitlines()
    chunks = []
    start = 0
    while start < len(lines):
        end = min(start + chunk_size, len(lines))
        chunk = "\n".join(lines[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Ejemplo de uso:
all_chunks = []
for filename, content in files_content.items():
    chunks = split_into_chunks(content, chunk_size=50, overlap=10)
    for chunk in chunks:
        all_chunks.append({"text": chunk})



# Check the chunks generated:
print(all_chunks[0]["text"])

In [ ]:
from datasets import Dataset

# all_chunks = [{"text": "code..."}, {"text": "code..."}] and we need to convert it into a HF Dataset structure:
dataset = Dataset.from_list(all_chunks)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 13438
})


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,   # puedes ajustarlo
        padding="max_length"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# Check the tokenized dataset
tokenized_dataset[0]['input_ids']

Map (num_proc=4):   0%|          | 0/13438 [00:00<?, ? examples/s]

In [ ]:
# Create the config arguments:

training_args = TrainingArguments(
    output_dir="output_tinyllama_code",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to=[]
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,
    label_names=["labels"])

trainer.train()

/tmp/ipython-input-502089872.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,2.063500
100,0.965600
150,0.879000
200,0.865900
250,0.824400
300,0.814400
350,0.800300
400,0.782700
450,0.778300
500,0.791800


TrainOutput(global_step=1260, training_loss=0.8446915872513302, metrics={'train_runtime': 6524.2052, 'train_samples_per_second': 6.179, 'train_steps_per_second': 0.193, 'total_flos': 1.2839777780052787e+17, 'train_loss': 0.8446915872513302, 'epoch': 3.0})

In [ ]:
from peft import PeftModel

# model es tu PeftModelForCausalLM entrenado
output_dir = "./tinyllama-lora-finetuned"

model.save_pretrained(output_dir)


from transformers import AutoTokenizer

tokenizer.save_pretrained(output_dir)


('./tinyllama-lora-finetuned/tokenizer_config.json',
 './tinyllama-lora-finetuned/special_tokens_map.json',
 './tinyllama-lora-finetuned/chat_template.jinja',
 './tinyllama-lora-finetuned/tokenizer.model',
 './tinyllama-lora-finetuned/added_tokens.json',
 './tinyllama-lora-finetuned/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Carga modelo base
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16, device_map="auto")

# Aplica LoRA
model = PeftModel.from_pretrained(base_model, "./tinyllama-lora-finetuned")

# Carga tokenizer
tokenizer = AutoTokenizer.from_pretrained("./tinyllama-lora-finetuned")


In [ ]:
from google.colab import files
files.download("tinyllama-lora-finetuned")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r tinyllama-lora-finetuned.zip tinyllama-lora-finetuned
!cp tinyllama-lora-finetuned.zip /content/drive/MyDrive/tinyllama-lora-finetuned.zip


updating: tinyllama-lora-finetuned/ (stored 0%)
updating: tinyllama-lora-finetuned/adapter_config.json (deflated 56%)
updating: tinyllama-lora-finetuned/chat_template.jinja (deflated 60%)
updating: tinyllama-lora-finetuned/tokenizer.model (deflated 55%)
updating: tinyllama-lora-finetuned/adapter_model.safetensors (deflated 8%)
updating: tinyllama-lora-finetuned/special_tokens_map.json (deflated 79%)
updating: tinyllama-lora-finetuned/tokenizer.json (deflated 85%)
updating: tinyllama-lora-finetuned/README.md (deflated 66%)
updating: tinyllama-lora-finetuned/tokenizer_config.json (deflated 69%)
cp: cannot create regular file '/content/drive/MyDrive/tinyllama-lora-finetuned.zip': No such file or directory


In [ ]:
def add_labels(examples):
    examples['labels'] = examples['input_ids'][:]
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True, num_proc=4)

# Verify the dataset structure
print(tokenized_dataset)

Map (num_proc=4):   0%|          | 0/13438 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 13438
})
